In [1]:
from dotenv import load_dotenv
import os, json
import requests

load_dotenv()
api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)
ibm_cloud_iam_url = os.getenv("IAM_IBM_CLOUD_URL", None)

creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    "decoding_method":"greedy",
    "max_new_tokens":3000,
    "min_new_tokens":1,
    # "temperature":0.1,
    "top_k":50,
    "top_p":1,
     "repetition_penalty": 1.1,
    "stop_sequences":["```","<|user|>","<|assistant|>"],
}

In [ ]:
# Prepare the payload and headers
payload = {
    "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
    "apikey": api_key
}
headers = {
    'Content-Type': "application/x-www-form-urlencoded"
}

# Make a POST request while ignoring SSL certificate verification
try:
    response = requests.post(f"https://{ibm_cloud_iam_url}/identity/token", data=payload, headers=headers, verify=False)
    
    # Check if the request was successful
    response.raise_for_status()

    # Parse the JSON response
    decoded_json = response.json()
    access_token = decoded_json["access_token"]
    # print(f"Access Token: {access_token}")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

In [3]:
import requests

GRANITE_3B_CODE_INSTRUCT = "ibm/granite-3b-code-instruct"
GRANITE_8B_CODE_INSTRUCT = "ibm/granite-8b-code-instruct"
GRANITE_20B_CODE_INSTRUCT = "ibm/granite-20b-code-instruct"
GRANITE_34B_CODE_INSTRUCT = "ibm/granite-34b-code-instruct"
GRANITE_3_8B_INSTRUCT = "ibm/granite-3-8b-instruct"

def watsonx_code_gen(prompt,model_id):

    url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation_stream?version=2023-05-29"

    body = {
        "input": prompt,
        "parameters": params,
        "model_id": model_id,
        "project_id": project_id
    }

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.post(
        url,
        headers=headers,
        json=body,
        stream=True
    )

    if response.status_code != 200:
        raise Exception("Non-200 response: " + str(response.text))

    answer = ""
    # Stream the response
    for line in response.iter_lines():
        if line:  # Ensure the line is not empty
            decoded_line = line.decode("utf-8").strip()
            
            # Check if the line starts with "data: "
            if decoded_line.startswith("data: "):
                json_data = decoded_line[len("data: "):]  # Remove the "data: " prefix
                
                try:
                    # Attempt to load the JSON data
                    data = json.loads(json_data)
                    generated_text = data.get("results", "")
                    answer += generated_text[0]["generated_text"]
                    # print(generated_text[0]["generated_text"],end="")  # Print or process the generated text as needed
                except json.JSONDecodeError:
                    print("Failed to decode JSON:", json_data)
                except Exception as e:
                    print("An error occurred:", e)

    return answer


In [4]:
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
import os
from dotenv import load_dotenv


load_dotenv()
api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)

creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    GenParams.DECODING_METHOD:"greedy",
    GenParams.MAX_NEW_TOKENS:4000,
    GenParams.MIN_NEW_TOKENS:1,
    # "temperature":0.1,
    GenParams.TOP_K:50,
    GenParams.TOP_P:1,
    GenParams.STOP_SEQUENCES:["```","[/INST]","<|user|>","<|endoftext|>","<|assistant|>"],
}

model = ModelInference(model_id="ibm/granite-3-8b-instruct", credentials=creds, params=params, project_id=project_id)

In [25]:
model_id = GRANITE_3_8B_INSTRUCT
language = "cpp"
documentframework = "doxygen"
testframework = "xunit"

In [10]:
prompt = f"""<|user|>
Please generate a sample {language} code:
-give more detail and complex logic.
-generate the code only.
-dont generate explanation.
<|assistant|>
```{language}
"""

sample_code = model.generate_text(prompt).replace("```","")
print(sample_code)

#include <iostream>
#include <vector>
#include <algorithm>

struct Person {
    std::string name;
    int age;
};

bool compareByAge(const Person& a, const Person& b) {
    return a.age < b.age;
}

int main() {
    std::vector<Person> people = {
        {"Alice", 30},
        {"Bob", 25},
        {"Charlie", 35},
        {"Diana", 28},
        {"Eve", 40}
    };

    std::sort(people.begin(), people.end(), compareByAge);

    for (const auto& person : people) {
        std::cout << person.name << ": " << person.age << std::endl;
    }

    return 0;
}



In [11]:
sample_code = """#include <iostream>
#include <vector>
#include <algorithm>

struct Person {
    std::string name;
    int age;
};

bool compareByAge(const Person& a, const Person& b) {
    return a.age < b.age;
}

int main() {
    std::vector<Person> people = {
        {"Alice", 30},
        {"Bob", 25},
        {"Charlie", 35},
        {"Diana", 28},
        {"Eve", 40}
    };

    std::sort(people.begin(), people.end(), compareByAge);

    for (const auto& person : people) {
        std::cout << person.name << ": " << person.age << std::endl;
    }

    return 0;
}
"""

bad_code = """#include <iostream>
#include <vector>

struct Person {
    std::string name;
    int age;
};

bool compareByAge(const Person& a, const Person& b) {
    return a.age < b.age;
}

int main() {
    std::sort(people.begin(), , compareByAge);

    for (const auto& person : p": " << person.age << std::endl;
    }

    return 0;
}"""

gnerate explanation

In [12]:
prompt = f"""<|user|>
Please provide a detailed explanation for the following {language} code:
-ensure have new line after every full stop.

###
code: `{sample_code}`
<|assistant|>
explanation:
"""

answer = model.generate_text(prompt)
print(answer)


This C++ code is a simple program that sorts a vector of `Person` structs by age and then prints out each person's name and age. Here's a step-by-step explanation:

1. The code starts by including the necessary libraries: `<iostream>` for input/output operations, `<vector>` for dynamic arrays, and `<algorithm>` for sorting functions.

2. A `struct` named `Person` is defined with two members: `name` (a string) and `age` (an integer).

3. A function `compareByAge` is defined, which takes two `Person` structs as arguments and returns a boolean value. This function is used to compare the ages of two people and determine their order in the sorted vector.

4. In the `main` function, a vector of `Person` structs is created and initialized with five people. Each person is defined as a pair of a name (a string) and an age (an integer).

5. The `std::sort` function is called with the `people` vector, the `compareByAge` function, and the range of the vector (from the beginning to the end). This 

generate javadoc

In [15]:
prompt = f"""<|user|>
generate {documentframework} document for the {language} provided.
-generate the {language} code only.
{language}: `{sample_code}`
<|assistant|>
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


/**
 * @file main.cpp
 * @brief A simple C++ program that sorts a vector of Person structs by age.
 *
 * This program defines a Person struct with a name and age, and a comparison
 * function to sort a vector of Person structs by age. The main function creates
 * a vector of Person structs, sorts it using the comparison function, and then
 * prints out the names and ages of the people in the sorted vector.
 */

#include <iostream>
#include <vector>
#include <algorithm>

/**
 * @struct Person
 * @brief A struct representing a person with a name and age.
 */
struct Person {
    std::string name;
    int age;
};

/**
 * @brief A comparison function to sort a vector of Person structs by age.
 *
 * @param a The first Person struct to compare.
 * @param b The second Person struct to compare.
 * @return true if a.age is less than b.age, false otherwise.
 */
bool compareByAge(const Person& a, const Person& b) {
    return a.age < b.age;
}

int main() {
    std::vector<Person> people = {
     

score, review and improve

In [16]:
prompt = f"""<|user|>
you are {language} expert. you need to review the following {language} code. 
-please understand the {language} code provided in detail.
-please review the code in completeness, logic, performance and quality.
-make it more object oriented.
-give a score out of 10 for the quality of code.
-please provide constructive suggestion in detail.
-generate the review and suggestion only.
{language} code: `{sample_code}`
-DONT generate the improve code directly.
<|assistant|>
score and review:"""

suggestion = model.generate_text(prompt)
print(suggestion)



The provided C++ code is a simple program that sorts a vector of Person structs by age using the std::sort function and a custom comparison function. The code is well-structured, easy to understand, and follows good coding practices. However, it can be made more object-oriented and efficient.

Here's a detailed review:

1. Completeness: The code is complete and fulfills its purpose of sorting a vector of Person structs by age.
2. Logic: The logic is clear and straightforward. The custom comparison function compareByAge is used correctly to sort the vector.
3. Performance: The code has a time complexity of O(n log n) due to the use of std::sort, which is efficient for sorting operations. However, the code could be made more efficient by avoiding the creation of a temporary vector when sorting.
4. Quality: The code is of good quality, but it could be improved by making it more object-oriented and adding error handling.

Suggestions for improvement:

1. Object-oriented design: The code 

In [18]:
prompt = f"""<|user|>
improve the {language} provided base on the review provided.
-understand the review suggestion in detail.
-adopt the review suggestion if applicable.
-dont generate explanation.
###
original {language} code: `{sample_code}`
review suggestion: `{suggestion}`
<|assistant|>
improved {language}
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


#include <iostream>
#include <vector>
#include <algorithm>

class Person {
public:
    Person(const std::string& name, int age) : name(name), age(age) {}

    const std::string& getName() const { return name; }
    int getAge() const { return age; }

    bool operator<(const Person& other) const {
        return age < other.age;
    }

private:
    std::string name;
    int age;
};

int main() {
    std::vector<Person> people = {
        {"Alice", 30},
        {"Bob", 25},
        {"Charlie", 35},
        {"Diana", 28},
        {"Eve", 40}
    };

    std::sort(people.begin(), people.end());

    for (const auto& person : people) {
        std::cout << person.getName() << ": " << person.getAge() << std::endl;
    }

    return 0;
}



fix bad code

In [19]:
prompt = f"""<|user|>
fix the {language} provided.
-understand the review suggestion in detail.
-identify if any problem.
-fix the problem if any.
-add explanation as inline comment.

###
{language}: `{bad_code}`
<|assistant|>
fixed version with inline comment at the point it fix.
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


#include <iostream>
#include <vector>

struct Person {
    std::string name;
    int age;
};

bool compareByAge(const Person& a, const Person& b) {
    return a.age < b.age;
}

int main() {
    std::vector<Person> people = {{"Alice", 30}, {"Bob", 25}, {"Charlie", 35}};
    std::sort(people.begin(), people.end(), compareByAge); // fixed here

    for (const auto& person : people) {
        std::cout << "Name: " << person.name << ", Age: " << person.age << std::endl;
    }

    return 0;
}


The issue with the original code was that the `std::sort` function was not provided with the correct range for sorting. The range should be from the beginning of the vector to the end of the vector, which is `people.begin()` and `people.end()`, respectively. The original code was missing the second argument, which caused a compilation error.

The fixed code now correctly sorts the `people` vector by age using the `compareByAge` function and then prints the name and age of each person in the sorted v

generate diagram

In [30]:
prompt = f"""<|user|>
generate diagram for the {language} code provided.
-understand the review suggestion in detail.
-use simple name for node.
-generate in mermaid.
-dont repeating.
-dont generation explanation.
{language}: `{sample_code}`
<|assistant|>
```mermaid"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


graph TD
    A[main] --> B[initialize people vector]
    B --> C[define compareByAge function]
    C --> D[sort people vector by age]
    D --> E[print sorted people vector]
    E --> F[return 0]



conversion

In [31]:
target_language = "python"

prompt = f"""<|user|>
rewrite the {language} code provided to {target_language}.
-understand the code provided in detail.
-understand how to map technology and syntax to the target {target_language} langauge.
-dont generation explanation.
{language}: `{sample_code}`
<|assistant|>
```{target_language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

def compare_by_age(a, b):
    return a.age < b.age

people = [
    Person("Alice", 30),
    Person("Bob", 25),
    Person("Charlie", 35),
    Person("Diana", 28),
    Person("Eve", 40)
]

people.sort(key=functools.cmp_to_key(compare_by_age))

for person in people:
    print(person.name, ":", person.age)



generate api definition

In [32]:
prompt = f"""<|user|>
generate a API definition in swagger for the {language} code provided.
-dont generate explanation.
{language}: `{sample_code}`
<|assistant|>
```yaml
"""

answer = model.generate_text(prompt).replace("```","")
print(answer)

openapi: 3.0.0
info:
  title: Person Sorting API
  version: 1.0.0
servers:
  - url: http://api.example.com/people
paths:
  /sort:
    post:
      summary: Sort people by age
      requestBody:
        content:
          application/json:
            schema:
              type: array
              items:
                $ref: '#/components/schemas/Person'
      responses:
        '200':
          description: Successful response
          content:
            application/json:
              schema:
                type: array
                items:
                  $ref: '#/components/schemas/Person'
components:
  schemas:
    Person:
      type: object
      properties:
        name:
          type: string
        age:
          type: integer



generate test case

In [28]:
prompt = f"""<|user|>
generate Test Cases in {testframework} for the {language} code provided.
-focus on boundary cases.
-focus on exception handling
-generate the test only.
{language}: `{sample_code}`
<|assistant|>
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


#include <iostream>
#include <vector>
#include <algorithm>
#include <stdexcept>
#include <xunit.hpp>

struct Person {
    std::string name;
    int age;
};

bool compareByAge(const Person& a, const Person& b) {
    return a.age < b.age;
}

TEST_CASE("Sort people by age", "[sort]") {
    std::vector<Person> people = {
        {"Alice", 30},
        {"Bob", 25},
        {"Charlie", 35},
        {"Diana", 28},
        {"Eve", 40}
    };

    std::sort(people.begin(), people.end(), compareByAge);

    REQUIRE(people[0].name == "Bob");
    REQUIRE(people[1].name == "Diana");
    REQUIRE(people[2].name == "Alice");
    REQUIRE(people[3].name == "Charlie");
    REQUIRE(people[4].name == "Eve");
}

TEST_CASE("Sort people by age with equal ages", "[sort]") {
    std::vector<Person> people = {
        {"Alice", 30},
        {"Bob", 30},
        {"Charlie", 35},
        {"Diana", 30},
        {"Eve", 40}
    };

    std::sort(people.begin(), people.end(), compareByAge);

    REQUIRE(people[0].na